In [1]:
'''
以下代码将实现以下功能

1. 自定义Siamese单支网络（具体代码实现参考 from models.base_model import base_model_sigmoid）
2. 计算双支网络输出特征差
3. 针对特征差,通过sigmoid激活函数计算唯一输出值
4. 根据输出值大小，判断样本对是否匹配
4. 定义输入样本对生成器（具体代码实现参考 from utils.images_generator import generator）
5. 通过model.fit_generator训练模型，保存模型
'''

'\n以下代码将实现以下功能\n\n1. 自定义Siamese单支网络（具体代码实现参考 from models.base_model import base_model_sigmoid）\n2. 计算双支网络输出特征差\n3. 针对特征差,通过sigmoid激活函数计算唯一输出值\n4. 根据输出值大小，判断样本对是否匹配\n4. 定义输入样本对生成器（具体代码实现参考 from utils.images_generator import generator）\n5. 通过model.fit_generator训练模型，保存模型\n'

In [1]:
import os
import cv2
import csv
import tensorflow as tf
import numpy as np
import random
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.optimizers import SGD,RMSprop
from keras.models import Model, Sequential,load_model
from keras.layers import Input,Concatenate, Add,Subtract,Lambda,Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam,RMSprop,SGD

Using TensorFlow backend.


In [2]:
#set GPU to memory-growth mode

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
from models.base_model import base_model_sigmoid
from utils.images_generator import generator

input_shape=(160,160,3)
base_network=base_model_sigmoid(input_shape)

base_network.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 166, 166, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        1568      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 16)        528       
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 16)        64        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 64)        9280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 64)        256 

In [4]:
left_input = Input(input_shape)
right_input = Input(input_shape)

encoded_l = base_network(left_input)
encoded_r = base_network(right_input)

distance = Subtract()([encoded_l,encoded_r])
abs_distance = Lambda(lambda x: K.abs(x)) (distance)

prediction = Dense(1,activation='sigmoid')(abs_distance)

siamese_net = Model(input=[left_input,right_input],output=prediction)

siamese_net.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          241552      input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
subtract_1 (Subtract)           (None, 128)          0           model_1[1][0]              

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [5]:
#download training images pair
train_csv_path='/root/tf-2.0/Siamese_Face_Detection/data/FaceV5_160_train.csv'
test_csv_path='/root/tf-2.0/Siamese_Face_Detection/data/FaceV5_160_test.csv'

rows_train = csv.reader(open(train_csv_path, 'r'), delimiter=',')
imgs_train = list(rows_train)

rows_test=csv.reader(open(test_csv_path, 'r'), delimiter=',')
imgs_test = list(rows_test)

#define iterations
epochs = 5
batch_size = 20
train_iterations = len(imgs_train)//20
test_iterations = len(imgs_test)//20

#choose the proper optimizer
initial_lr=0.001
decay_rate=initial_lr/epochs
sgd=SGD(learning_rate=initial_lr,decay=decay_rate,momentum=0.9)
rms=RMSprop()
adam = Adam(0.0001)

In [8]:
from utils.images_generator import generator

model_result = '/root/tf-2.0/Siamese_Face_Detection/h5/'
#define training iterator
train_data=generator(imgs_train, batch_size,input_shape)
valid_data=validation_data = generator(imgs_test,batch_size,input_shape)

checkpoint = ModelCheckpoint(filepath=model_result + 'Siamese_sigmoid_{epoch:03d}.h5', verbose=1)
siamese_net.compile(loss="binary_crossentropy",optimizer=rms, metrics=['binary_accuracy'])

history=siamese_net.fit_generator(train_data,
                            steps_per_epoch = train_iterations,
                            epochs = epochs,
                            validation_data = valid_data,
                            validation_steps = test_iterations,
                            callbacks=[checkpoint])




Epoch 1/5
3000/3000 [==============================] - 184s 61ms/step - loss: 0.2125 - binary_accuracy: 0.9233 - val_loss: 0.1187 - val_binary_accuracy: 0.9341

Epoch 00001: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_sigmoid_001.h5
Epoch 2/5
3000/3000 [==============================] - 180s 60ms/step - loss: 0.1669 - binary_accuracy: 0.9304 - val_loss: 0.1171 - val_binary_accuracy: 0.9341

Epoch 00002: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_sigmoid_002.h5
Epoch 3/5
3000/3000 [==============================] - 180s 60ms/step - loss: 0.1434 - binary_accuracy: 0.9401 - val_loss: 0.0892 - val_binary_accuracy: 0.9519

Epoch 00003: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_sigmoid_003.h5
Epoch 4/5
3000/3000 [==============================] - 183s 61ms/step - loss: 0.1287 - binary_accuracy: 0.9468 - val_loss: 0.0119 - val_binary_accuracy: 0.9526

Epoch 00004: saving model to /root/tf-2.0/Siamese_Face_Detection/h5/Siamese_sigmoid

In [9]:
siamese_net.save('/root/tf-2.0/Siamese_Face_Detection/h5/Siamese_sigmoid.h5')